# Pablo the Thymio visits its first museum 

Today, Pablo wants to go to the museum. Since he's never been in a art museum, he decide to go to the famous "Aseba Museum"! 
The lady at the entrance gives him a map of the museum and thymio decides to study it before visiting.
He goes in the cafeteria, order a cup of coffee and take his 4-colors pen.
He circles in red all the painting he wants to see, and draw a green path to see them all.

The adventure now begins:


**Libraries to import**

In [1]:
import cv2
import time
import os
import sys
import serial
import src
import math

import numpy as np
from math import *

%matplotlib notebook 
import matplotlib.pyplot as plt
%matplotlib notebook
from matplotlib import colors
%matplotlib inline

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from tqdm import tqdm

import IPython.display as Disp
from ipywidgets import widgets

In [2]:
# Variable to know if Thymio has already been connected before or not (== 1 if it is the case)
try:
    CONNECC
except NameError:
    CONNECC = 0
    print('Thymio will be connected.')

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

# Print the path to Python3 executable
print(sys.executable)

Thymio will be connected.Thymio will be connected.

C:\Users\Oceane\Anaconda3\python.exeC:\Users\Oceane\Anaconda3\python.exe



In [3]:
if CONNECC == 0:
    CONNECC = 1
    th = Thymio.serial(port="COM10", refreshing_rate=0.1)
    time.sleep(1)
th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

**Magic numbers**

In [4]:
#Code for the occupancy grid 
obstacle_here = 1
painting_here = 2
goal_here = 3
start_here = 4
marker_here = 5

size_marker_cm = 6

**Define global variables**

In [5]:
#Size of the grid (defined in get_params())
global max_val_x, max_val_y
global list_delimitation #top left and bottom right corners [pixel]

**Initialize the class Thymio, with position, sensor values etc**

In [6]:
class T:
    def __init__(self, x, y, vel): # x and y are in cm, theta in degrees
        self.x = x
        self.y = y
        self.theta = pi
        self.ground = [0,0]
        self.front = [0,0,0,0,0]
        self.vel = vel
    
    def set_pos(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta
    
    def get_ground(self):
        self.ground = th["prox.ground.delta"] # à déterminer s'il vaut mieux choisir ground.ambiant, 
                                              # ground.delta
        return self.ground                    # ou bien encore ground.... 
                                              #(le dernier des trois possibles, voir les variables Aseba)
        
    def get_front(self):
        self.front = th["prox.horizontal"][0:4]
        return self.front
        
    def pos(self):
        return [float(self.x), float(self.y), float(self.theta)]
    
    def move(self, u): # soit simplement set la vitesse des roues, 
                       # possible de faire l'odometry ici aussi je pense
        
        # Move thymio depending on the control inputs u
        th.set_var("motor.left.target", u[0])
        the.set_var("motor.right.target", u[1])
        
        
    def __repr__(self): # called when print(Thymio) is used
        return "(x, y, theta) = (%.2f, %.2f, %.1f)" %(self.x, self.y, self.theta)

# -----------------------------------------------VISION-----------------------------------------------

The function `detect_images()` allows to detect an image (`template`) within another (`img`). We use this function to return the position of the center of each painting in the map. After trying the different methods for matching with the template, we found that the square difference normed seemed to work best.

In [7]:
def detect_image(img, template, display_images):
    """
    Find template in img and draw a rectangle on it
        if display_images, display the image
    :return: coordinates of the center of the template in the image
    """
    w, h = template.shape[::-1]
    
    method = cv2.TM_SQDIFF_NORMED

    # Apply template Matching
    res = cv2.matchTemplate(img,template,method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    #take minimum (due to the choice of the method)
    top_left = min_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    #draw rectangle in img 
    cv2.rectangle(img,top_left, bottom_right, (0,0,255), 7)

    if display_images:
        plt.figure()
        ax4 = plt.subplot(121),plt.imshow(res, cmap = 'gray')
        plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
        ax5 = plt.subplot(122),plt.imshow(img, cmap = 'gray')
        plt.title('Detected picture'), plt.xticks([]), plt.yticks([])
        plt.suptitle(method)
        
        plt.show()
        
    center = [(top_left[0] + bottom_right[0])/2, (top_left[1] + bottom_right[1])/2]
    return center #coordinates of the center of the image.

`detect_multiple` uses the same principle as `detect_image`, except it can detect several occurences of the same template in the image. We use this function to detect the corners of the map, as well as the obstacles. Returns a list of the coordinates of the top left corners of each occurence.

In [8]:
def detect_multiple(name_img, img, template, threshold, display_images):
    """
    Detect all template in image and draw a rectangle on each one of them 
        if display_images, display the image
    :return: list of coordinates of each template's top left 
    """
    
    img_gray = img
    img_rgb = cv2.imread(name_img, cv2.IMREAD_COLOR)
    
    w, h = template.shape[::-1]

    res = cv2.matchTemplate(img_gray,template,cv2.TM_SQDIFF_NORMED)

    #take all coordinate where template is detected
    loc = np.where(res <= threshold)
    
    #Since each template might be detected several time, we need to make sure we have one coordinate by template
    
    #take 1st point, init the precedent point and create a list of point
    pt_prec = [loc[1][0], loc[0][0]]
    list_pt = [pt_prec] 
    
    #draw rectangle for first point in img_rgb
    cv2.rectangle(img_rgb, (pt_prec[0], pt_prec[1]), (int(pt_prec[0] + w), int(pt_prec[1] + h)), (255,0,0), 5)
    
    #list of extra point we don't want
    pt_notchoosen = []

    #Check if point in loc is a new obstacle or not
    for pt in zip(*loc[::-1]):
        
        #if not too close of the previous choosen point, might be a new obstacle
        if (abs(pt[0] - pt_prec[0]) > 1) or (abs(pt[1] - pt_prec[1]) > 1) :
            same_obstacle = 0
            
            #if too close of not wanted point, not a new obstacle, we don't want it
            for pt_not in pt_notchoosen:
                if (abs(pt[0] - pt_not[0]) <=10) and (abs(pt[1] - pt_not[1]) <= 10):
                    same_obstacle = 1
                    pt_notchoosen.append([pt[0], pt[1]])
                    break
            #if not too close, we add it to the list, draw a rectangle and update pt_prec
            if not same_obstacle : 
                list_pt.append([pt[0], pt[1]])
                cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (255,0,0), 5)
                pt_prec = pt
                
        #if too close of not wanted point, not a new obstacle, we don't want it        
        else:
            pt_notchoosen.append([pt[0], pt[1]])
                    
    
    #cv2.imwrite('res.png',img_rgb)
    if display_images:
        plt.figure()
        ax2 = plt.subplot(121),plt.imshow(res,cmap = 'gray')
        plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
        ax3 = plt.subplot(122),plt.imshow(img_rgb,cmap = 'gray')
        plt.title('Detected obstacles'), plt.xticks([]), plt.yticks([])
        
        plt.show()
        
    #list_pt contains the coordinates of the top left corners of all the objects    
    return list_pt, w, h

# Filtering

The function `filtering()` is taking an image, applying a bilateral filter on it. Then we decided to apply Otsu's thresholding to filter the image. This allows to filter out the checkerboard that is on the map, and also the symbols (paintings, start, goal) are clearer. This function returns the filtered image.



-Bw_photo is not used --> bilateralFilter not used ? *Change to used, if no need to detect marker --> ok* 
(ben si elle est used, c'est sur cette image qu'on applique le thresholding otsu.)


In [9]:
def filtering(name_img, display_images):
    """
    Filter the image using bilateral filter and otsu thersholding 
        if display_images, display the image
    :return: image filtered
    """
    photo = cv2.imread(name_img)
    bilateral = cv2.bilateralFilter(photo,9,75,75)
    
    #convert image in BW
    bw_photo = cv2.cvtColor(bilateral, cv2.COLOR_BGR2GRAY)

    #ret2,th2 = cv2.threshold(cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _,filtered_image = cv2.threshold(bw_photo,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    if display_images:
        plt.figure()
        ax = plt.subplot(121),plt.imshow(photo[:,:,::-1], cmap = 'gray')
        plt.title('Original'), plt.xticks([]), plt.yticks([])
        ax1 = plt.subplot(122),plt.imshow(filtered_image, cmap = 'gray')
        plt.title('Filtered image'), plt.xticks([]), plt.yticks([])
        
        plt.show()
        
    return filtered_image

# ----------------------------------------------Parameters----------------------------------------------

In `get_params()`, we can set all the parameters : the name of the images files (the map, obstacles and corners are in the folder 'structure', and the paintings, start, goal are in the folder 'oeuvres'). 
The grid parameters can also be changed. It is the number of squares in x and y (`max_val_x`and `max_val_y`).

All images are then filtered with `filtering()`. We use `detect_image`and `detect_multiple` in order to get the :
    - list of corners (top left coordinates)
    - list of obstacles (top left coordinates)
    - list of paintings (center coordinates)
    - start and stop positions (center coordinates)
    
We also return the width and height of the corners and obstacles, that we will use later in `get_rectangle` and `create_fill_grid`.

In [10]:
def get_param():
    global max_val_x, max_val_y
    
    name_map ='structure/cartecoindamier.jpg'
    name_picture = ['oeuvres/triangle.png', 'oeuvres/nuage.png', 'oeuvres/coeur.png']
    name_obstacle = 'structure/obstacle.png'
    name_corner = 'structure/coin.png'
    
    #departure and goal symbols
    name_start = 'oeuvres/thymio.png'
    name_goal = 'oeuvres/etoile.png'

    #G R I D parameters
    max_val_x = 20
    max_val_y = 14
    
    mapim = cv2.imread(name_map,0)
    obstacle = cv2.imread(name_obstacle,0)
    corner = cv2.imread(name_corner,0)
    start = cv2.imread(name_start,0)
    goal = cv2.imread(name_goal,0)
 
    display_images = False

    #filter images
    map_filtered = filtering(name_map, display_images)
    obstacle_filtered = filtering(name_obstacle, display_images)
    corner_filtered = filtering(name_corner, display_images)
    start_filtered = filtering(name_start, display_images)
    goal_filtered = filtering(name_goal, display_images)
    
    picture_filtered = []
    list_paintings = []
    #get the list of the center coordinates of all paintings
    for i in range(len(name_picture)):
        picture_filtered.append(filtering(name_picture[i], display_images))
        list_paintings.append(detect_image(map_filtered, picture_filtered[i], display_images))

    #get list of coordinates of obstacles and corners
    list_obstacles,w_obst,h_obst = detect_multiple(name_map, map_filtered, obstacle_filtered, 0.8 , display_images)
    list_corners, w_corner, h_corner = detect_multiple(name_map, map_filtered, corner_filtered, 0.2, display_images)
    
    #get coordinates of the start and stop position
    start = detect_image(map_filtered, start_filtered, display_images)
    goal = detect_image(map_filtered, goal_filtered, display_images)
    
    return list_paintings, list_obstacles, w_obst, h_obst, list_corners, w_corner, h_corner, start, goal

# ------------------------------------------------GRID------------------------------------------------

In [11]:
def get_rectangle(list_corners, w_corner, h_corner):
    #take the smallest rectangle inside the corners
    sort_corners = sorted(list_corners) #sorts points according to x
    x1 = max(sort_corners[0][0], sort_corners[1][0])
    x2 = min(sort_corners[2][0], sort_corners[3][0]) + w_corner

    sort_corners = sorted(list_corners , key=lambda k: [k[1], k[0]])
    y1 = max(sort_corners[0][1], sort_corners[1][1])
    y2 = min(sort_corners[2][1], sort_corners[3][1]) + h_corner

    return [[x1,y1],[x2,y2]]

In [12]:
def create_visual_grid(): #allows to display the grid and Astar.
    fig, ax = plt.subplots(figsize=(7,7))

    major_ticks = np.arange(0, max_val_x+1, 10)
    minor_ticks = np.arange(0, max_val_x+1, 1)
    ax.set_xticks(major_ticks)
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)
    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_ylim([-1,max_val_y])
    ax.set_xlim([-1,max_val_x])
    ax.grid(True)
    return fig, ax

**Conversion functions**

In [13]:
def convert_px_to_grid(x,y):
    x_grid = max_val_x*(x - list_delimitation[0][0])/(list_delimitation[1][0] - list_delimitation[0][0])
    y_grid = max_val_y - max_val_y*(y - list_delimitation[0][1])/(list_delimitation[1][1] - list_delimitation[0][1])
    return [x_grid,y_grid]

In [14]:
def convert_cm_to_grid(x,y):
    x_grid = floor(x/size_marker_cm)
    y_grid = floor(y/size_marker_cm)
    return [x_grid,y_grid]

In [15]:
def convert_grid_to_cm(x,y):
    x_cm = x*size_marker_cm
    y_cm = y*size_marker_cm
    return [x_cm, y_cm]

Create The grid and fill it

Thanks to the lists of corners, obstacles, paintings, and the start and stop coordinates, we can now fill the occupancy grid. The magic numbers defined beforehand will help filling it. A `safe_zone` is included : for us, the 'center' of the Thymio is the circle in the middle of its wheels. So, we add a safe zone of 8cm around each obstacle to make sure the Thymio will not drive on an obstacle.

In [16]:
def create_fill_grid():
    global list_delimitation
    #Get coordinate
    list_paintings, list_obstacles, w_obst, h_obst, list_corners, w_corner, h_corner, start, goal = get_param()
    
    #Create visual grid
    fig, ax = create_visual_grid()
    
    # Creating the occupancy grid
    occupancy_grid = np.zeros((max_val_x, max_val_y)) # Create an empty grid of max_val_x X max_val_y
 
    cmap = colors.ListedColormap(['white', 'black', 'red', 'blue', 'yellow', 'pink']) # Select the colors: Black = obstacles, 
                                                            # Red = paintings, blue = goal, yellow = departure??

    #Get environnement delimitations
    list_delimitation = get_rectangle(list_corners, w_corner, h_corner)
    
    #thymio grid size
    half_thymio = 83 #[mm]
    size_sheet = [841, 594] #[mm]
    safe_zone = [half_thymio * max_val_x/size_sheet[0], half_thymio * max_val_y/size_sheet[1]]
    
    
    for obs in list_obstacles:
        #Convert coordinate into grid coordinate and take into accompt the obstacle size and thymio size (safezone)
        obs_grid_tl = convert_px_to_grid(obs[0] - safe_zone[0], obs[1] + safe_zone[1])
        obs_grid_br = convert_px_to_grid(obs[0] + w_obst + safe_zone[0], obs[1] + h_obst - safe_zone[1])
        
        #check if out of the grid because of safe zone                
        for i in range(max(floor(obs_grid_tl[0]),0), min(ceil(obs_grid_br[0]), max_val_x)):
            for j in range(max(floor(obs_grid_br[1]), 0), min(ceil(obs_grid_tl[1]), max_val_y)):
                occupancy_grid[i,j] = obstacle_here

    list_paintings_grid = []
    #Convert coordinate into grid coordinate
    for paint in list_paintings:
        paint_grid = convert_px_to_grid(paint[0], paint[1])
        paint_grid = [round(paint_grid[0]), round(paint_grid[1])]
        list_paintings_grid.append(paint_grid)
        occupancy_grid[int(paint_grid[0]), int(paint_grid[1])] = painting_here

    goal_grid = convert_px_to_grid(goal[0], goal[1])
    occupancy_grid[int(goal_grid[0]), int(goal_grid[1])] = goal_here
    
    start_grid = convert_px_to_grid(start[0], start[1])
    occupancy_grid[int(start_grid[0]), int(start_grid[1])] = start_here

    # Displaying the map
    ax.imshow(occupancy_grid, cmap=cmap)
    
    return occupancy_grid, list_paintings_grid, start_grid, goal_grid, fig, ax
    

# ---------------------------------------------ASTAR---------------------------------------------

Here, we will use the function `get_movements_4n` because we do not want the Thymio to drive between two obstacles. We will proceed to a normal Astar algorithm.

In [17]:
def _get_movements_4n():
    """
    Get all possible 4-connectivity movements.
    :return: list of movements with cost [(dx, dy, movement_cost)]
    """
    return [(1, 0, 1.0),
            (0, 1, 1.0),
            (-1, 0, 1.0),
            (0, -1, 1.0)]

def reconstruct_path(cameFrom, current):
    """
    Recurrently reconstructs the path from start node to the current node
    :param cameFrom: map (dictionary) containing for each node n the node immediately 
                     preceding it on the cheapest path from start to n 
                     currently known.
    :param current: current node (x, y)
    :return: list of nodes from start to current node
    """
    total_path = [current]
    while current in cameFrom.keys():
        # Add where the current node came from to the start of the list
        total_path.insert(0, cameFrom[current]) 
        current=cameFrom[current]
    return total_path

def A_Star(start, goal, h, coords, occupancy_grid):
    """
    A* for 2D occupancy grid. Finds a path from start to goal.
    h is the heuristic function. h(n) estimates the cost to reach goal from node n.
    :param start: start node (x, y)
    :param goal_m: goal node (x, y)
    :param occupancy_grid: the grid map
    :return: a tuple that contains: (the resulting path in meters, the resulting path in data array indices)
    """

    for point in [start, goal]: 
        assert point[0]>=0 and point[0]<max_val_x, "start or end goal not contained in the map (x)"
        assert point[1]>=0 and point[1]<max_val_y, "start or end goal not contained in the map (y)"
    
    # check if start and goal nodes correspond to free spaces
    if occupancy_grid[start[0], start[1]] == 1:
        raise Exception('Start node is not traversable')

    if occupancy_grid[goal[0], goal[1]] == 1:
        raise Exception('Goal node is not traversable')
    
    # get possible movements
    movements = _get_movements_4n()
    
    
    # The set of visited nodes that need to be (re-)expanded, i.e. for which the neighbors need to be explored
    # Initially, only the start node is known.
    openSet = [start]
    
    # The set of visited nodes that no longer need to be expanded.
    closedSet = []

    # For node n, cameFrom[n] is the node immediately preceding it on the cheapest path from start to n currently known.
    cameFrom = dict()

    # For node n, gScore[n] is the cost of the cheapest path from start to n currently known.
    gScore = dict(zip(coords, [np.inf for x in range(len(coords))]))
    gScore[start] = 0

    # For node n, fScore[n] := gScore[n] + h(n). map with default value of Infinity
    fScore = dict(zip(coords, [np.inf for x in range(len(coords))]))
    fScore[start] = h[start]

    # while there are still elements to investigate
    while openSet != []:
        
        #the node in openSet having the lowest fScore[] value
        fScore_openSet = {key:val for (key,val) in fScore.items() if key in openSet}
        current = min(fScore_openSet, key=fScore_openSet.get)
        del fScore_openSet
        
        #If the goal is reached, reconstruct and return the obtained path
        if current == goal:
            return reconstruct_path(cameFrom, current), closedSet

        openSet.remove(current)
        closedSet.append(current)
        
        #for each neighbor of current:
        for dx, dy, deltacost in movements:
            
            neighbor = (current[0]+dx, current[1]+dy)
            
            # if the node is not in the map, skip
            if (neighbor[0] >= occupancy_grid.shape[0]) or (neighbor[1] >= occupancy_grid.shape[1]) or (neighbor[0] < 0) or (neighbor[1] < 0):
                continue
            
            # if the node is occupied or has already been visited, skip
            if (occupancy_grid[neighbor[0], neighbor[1]]==obstacle_here) or (neighbor in closedSet): 
                continue
                
            # d(current,neighbor) is the weight of the edge from current to neighbor
            # tentative_gScore is the distance from start to the neighbor through current
            tentative_gScore = gScore[current] + deltacost
            
            if neighbor not in openSet:
                openSet.append(neighbor)
                
            if tentative_gScore < gScore[neighbor]:
                # This path to neighbor is better than any previous one. Record it!
                cameFrom[neighbor] = current
                gScore[neighbor] = tentative_gScore
                fScore[neighbor] = gScore[neighbor] + h[neighbor]

    # Open set is empty but goal was never reached
    print("No path found to goal")
    return [], closedSet


The following function, `detect_stairs`, will detect if there is a way of "cutting" the path by driving diagonally. It takes the path returned by `A_star`, and transforms it by removing useless steps. This makes our navigation more efficient, the Thymio will reach its goal sooner.

In [18]:
def detect_stairs(path, occupancy_grid): 
    new_path = np.array([[path[0][0]], [path[1][0]]])
    #new_occ = np.rot90(occupancy_grid)
    new_occ = occupancy_grid
    
    for i in range (1,path.shape[1] - 2):        
        if (occupancy_grid[path[0][i+1]][path[1][i+1]]) == painting_here: #if painting keep point
            new_path = np.append(new_path, [[path[0][i+1]],[path[1][i+1]]], axis=1)
            
        else:
            if (path[0][i + 2] == path[0][i] + 1) and (path[1][i + 2] == path[1][i] + 1): #going up right
                occ1 = new_occ[path[0][i] + 1][path[1][i]]
                occ2 = new_occ[path[0][i]][path[1][i] + 1]
                if (occ1 == obstacle_here or occ2 == obstacle_here):
                    new_path = np.append(new_path, [[path[0][i]],[path[1][i]]], axis=1)
                    
            elif (path[0][i + 2] == path[0][i] - 1) and (path[1][i + 2] == path[1][i] + 1): #going up left
                occ1 = new_occ[path[0][i] - 1][path[1][i]]
                occ2 = new_occ[path[0][i]][path[1][i] + 1]
                if (occ1 == obstacle_here or occ2 == obstacle_here):
                    new_path = np.append(new_path, [[path[0][i]],[path[1][i]]], axis=1)
                    
            elif (path[0][i + 2] == path[0][i] + 1) and (path[1][i + 2] == path[1][i] - 1): #going down right
                occ1 = new_occ[path[0][i] + 1][path[1][i]]
                occ2 = new_occ[path[0][i]][path[1][i] - 1]
                if (occ1 == obstacle_here or occ2 == obstacle_here):
                    new_path = np.append(new_path, [[path[0][i]],[path[1][i]]], axis=1)

            elif (path[0][i + 2] == path[0][i] - 1) and (path[1][i + 2] == path[1][i] - 1): #going down left
                occ1 = new_occ[path[0][i] - 1][path[1][i]]
                occ2 = new_occ[path[0][i]][path[1][i] - 1]
                if (occ1 == obstacle_here or occ2 == obstacle_here):
                    new_path = np.append(new_path, [[path[0][i]],[path[1][i]]], axis=1)

            else:
                new_path = np.append(new_path, [[path[0][i]],[path[1][i]]], axis=1)
    
    new_path = np.append(new_path, [[path[0][-1]],[path[1][-1]]], axis=1)
    return new_path

`thymio_path` will first ask the user the order in which he/she want to visit the paintings. The user can enter the indices of the paintings, with or without spaces (for example `2 0 1`, `10`, etc). The function will return a path that starts from the `start` position detected by the vision. The path then goes through the paintings in the desired order, and ends up in the `goal` position detected by vision.

In [19]:
def thymio_path(list_paintings_grid, start_grid, goal_grid, fig_astar, ax_astar, occupancy_grid):

    # Define the start and end goal constant with entrance and exit of museum
    entrance = tuple([int(start_grid[0]), int(start_grid[1])]) #(0,0)
    exit = tuple([int(goal_grid[0]), int(goal_grid[1])]) #(20,10)


    # List of all coordinates in the grid
    x,y = np.mgrid[0:max_val_x:1, 0:max_val_y:1]
    pos = np.empty(x.shape + (2,))
    pos[:, :, 0] = x; pos[:, :, 1] = y
    pos = np.reshape(pos, (x.shape[0]*x.shape[1], 2))
    coords = list([(int(x[0]), int(x[1])) for x in pos])
    path = []
    visitedNodes = []

    p = input('Enter the order you want to to see the paintings in :')
    visiting_order = []
    i = 0
    x_t = []
    y_t = []
    pos_int = 0
    while i<len(p):
        if p[i].isdigit():
            visiting_order.append(int(p[i]))
            x_t.append(float(list_paintings_grid[int(p[i])][0]))
            y_t.append(float(list_paintings_grid[int(p[i])][1]))
        i = i + 1

    coords_goal = list([x_v, y_v] for (x_v,y_v) in zip(x_t,y_t))

    previous_pos = entrance #Start position
    goal_pos = coords_goal[0]

    for goal_pos in coords_goal:
        # Define the heuristic, here = distance to goal ignoring obstacles
        h = np.linalg.norm(pos - goal_pos, axis=-1)
        h = dict(zip(coords, h))
        goal_pos[0] = int(goal_pos[0])
        goal_pos[1] = int(goal_pos[1])

        # Run the A* algorithm #PUT ALL TOGETHER
        new_path, new_visitedNodes = A_Star(tuple(previous_pos), tuple(goal_pos), h, coords, occupancy_grid)
        if len(path) == 0:
            path = np.array(new_path).reshape(-1, 2).transpose()
        else:
            path = np.concatenate((path, np.array(new_path).reshape(-1, 2).transpose()), axis=1)

        if len(visitedNodes) == 0:
            visitedNodes = np.array(new_visitedNodes).reshape(-1, 2).transpose()
        else:
            visitedNodes = np.concatenate((visitedNodes, np.array(new_visitedNodes).reshape(-1, 2).transpose()), axis=1)

        previous_pos = goal_pos

    #####GO TO EXIT
    goal_pos = exit
    # Define the heuristic, here = distance to goal ignoring obstacles
    h = np.linalg.norm(pos - goal_pos, axis=-1)
    h = dict(zip(coords, h))

    # Run the A* algorithm
    new_path, new_visitedNodes = A_Star(tuple(previous_pos), goal_pos, h, coords, occupancy_grid) 
    if len(path) == 0:
            path = np.array(new_path).reshape(-1, 2).transpose()
    else:
        path = np.concatenate((path, np.array(new_path).reshape(-1, 2).transpose()), axis=1)

    if len(visitedNodes) == 0:
        visitedNodes = np.array(new_visitedNodes).reshape(-1, 2).transpose()
    else:
        visitedNodes = np.concatenate((visitedNodes, np.array(new_visitedNodes).reshape(-1, 2).transpose()), axis=1)
    
    path = detect_stairs(path, occupancy_grid)
    
    ##### Displaying the map
    plt.figure()
    #fig_astar, ax_astar = create_empty_plot()
    cmap = colors.ListedColormap(['white', 'black', 'red', 'blue', 'yellow', 'pink']) # Select the colors: Black = obstacles, Red = painting
    ax_astar.imshow(occupancy_grid.transpose(), cmap=cmap)

    # Plot the best path found and the list of visited nodes
    ax_astar.scatter(visitedNodes[0], visitedNodes[1], marker="o", color = 'orange')
    ax_astar.plot(path[0], path[1], marker="o", color = 'blue')
    ax_astar.scatter(entrance[0], entrance[1], marker="o", color = 'green', s=200)
    ax_astar.scatter(exit[0], exit[1], marker="o", color = 'purple', s=200)
    
    path2 = []
    #transform the path in a list of [x y]
    for i in range(0, path.shape[1]):
        path2.append([path[0][i], path[1][i]])
    
    return path2

**Create checkerboard pattern on the grid** 
For the filtering, we chose to print a map with a checkerboard pattern. We make sure that the vision does not detect this pattern and detects only the desired objects (paintings, etc).

We will use A0 format for our map. We want squares of 6cm, hence we need 20 x 14. The 20th 
square will be cropped, se we will move the corners.

In [20]:
def checkerboard(occupancy_grid):
    
    for x in range (0,max_val_x):
        for y in range (0,max_val_y):
            if not occupancy_grid[x,y]:
                if not ((x + y)%2):
                    occupancy_grid[x,y] = marker_here
            
    return occupancy_grid          
    


For the filtering part, we need to know the theoretical value that the ground sensor have according to their position. This is the purpose of `ground_sensor_theoretical`, that will be given a position and angle, and will return what each sensor (right and left) should return. `1` means that the sensor is on a grey square, `0` means whilte.

In [21]:
def ground_sensor_theoretical(theta, x, y, occupancy_grid): #theta in degrees, x, y, in cm
    pos_grid = convert_cm_to_grid(x,y)
    alpha = 9
    dist_sensor_center = 7.9
    
    if (x<0) or (y<0) or (pos_grid[0]>max_val_x) or (pos_grid[1]>max_val_y):
        print('ERROR : Thymio out of the grid')
        return
    pos_sensor_left = [dist_sensor_center*sin((90 - theta - alpha)*pi/180), 
                       dist_sensor_center*cos((90 - theta - alpha)*pi/180)]
    pos_sensor_right = [dist_sensor_center*sin((90 - theta + alpha)*pi/180),
                        dist_sensor_center*cos((90 - theta + alpha)*pi/180)]
    
    pos_sensor_left_grid = convert_cm_to_grid(x + pos_sensor_left[0], y + pos_sensor_left[1])
    pos_sensor_right_grid = convert_cm_to_grid(x + pos_sensor_right[0], y + pos_sensor_right[1])
    print("POS SENSOR GRID: ",pos_sensor_left_grid, pos_sensor_right_grid)
    
    left = occupancy_grid[int(pos_sensor_left_grid[0]), int(pos_sensor_left_grid[1])]
    right = occupancy_grid[int(pos_sensor_right_grid[0]), int(pos_sensor_right_grid[1])]
    return [left, right]

`convert_path_to_movements` is given the path calculated previousy. It will transform it into angles that the Thymio should have, and distances between each point and the next one. This function is needed before processing to making the Thymio move. The distance returned is in grid squares, and the angle in radians. HEEEYA

In [22]:
def theta_needed(p_x, p_y, n_x, n_y):
    theta = 0
    #1st quadrant
    if(n_x > p_x and n_y >= p_y):
        theta = np.arctan(abs(n_y-p_y)/abs(n_x-p_x))
    #2nd quadrant
    elif(n_x <= p_x and n_y > p_y):
        theta = pi/2 + np.arctan(abs(n_x-p_x)/abs(n_y-p_y))
    #3rd quadrant
    elif(n_x < p_x and n_y <= p_y):
        theta = pi + np.arctan(abs(n_y-p_y)/abs(n_x-p_x))
    #4th quadrant
    elif(n_x >= p_x and n_y < p_y):
        theta = 3*pi/2 + np.arctan(abs(n_x-p_x)/abs(n_y-p_y))
    
    return theta

`markers2reality` gives the position in real coordinates (cm) of the given markers (matrix coordinates). The checkerboard pattern has 6cm sided squares:

In [23]:
def markers2reality(mx, my):
    x = (2*mx + 1)*3
    y = (2*my + 1)*3
    return (x,y)

#### Kalmanfilter

Extended Kalman filter called at each iteration after Thymio's movement. See lecture on Uncertainties slides 33-end. Previous lecture on Localisation might be useful as well, odometry slides 19-20 and intro. to uncertainties slides 23-37.

In [24]:
class Kalmanfilter:
    def __init__(self, A, B, w, C1, C2, v, pred_mu, pred_Sigma, mu, Sigma):
        self.A = A # jacobian, constant
        self.B = B # odometry matrix, change with theta of Thymio
        self.w = w # displacement noise, constant
        self.C1 = C1 # measurement matrix measurements == predictions
        self.C2 = C2 # measurement matrix when measurements =/= predictions
        self.v = v # measurement noise, constant
        self.mu = mu # current mean, aka current state
        self.Sigma = 0 # current covariance matrix
        self.i = 0 #innovation
        
    def predict(self, u): # first step. u doit etre un array. SLIDE UNCERTAINTIES-39
        self.mu = np.matmul(self.A, self.mu) + np.matmul(self.B, u) #not sure about matmul
        self.Sigma = np.matmul(self.A, np.matmul(self.Sigma, np.transpose(self.A)))

    def innovation(self): # second step SLIDE UNCERTAINTIES-40
        #self.i = - np.matmul()
        pass
    def aposteriori(self): # final step
        pass

In [ ]:
#------MAIN SANS OBSTACLE AVOIDANCE AVEC ODOMETRY------
occupancy_grid, list_paintings_grid, start_grid, goal_grid, fig, ax= create_fill_grid()
occupancy_grid = checkerboard(occupancy_grid)
path = thymio_path(list_paintings_grid, start_grid, goal_grid, fig, ax, occupancy_grid)
start_cm = convert_grid_to_cm(start_grid[0],start_grid[1])
Pablo = T(start_cm[0], start_cm[1], 100)

Threshold_theta = 1 # Threshold in radians
Threshold_d = 0.5 # Threshold in cm
current_goal = path[0]
del path[0]
current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
p_x = Pablo.x
p_y = Pablo.y
VelLeft = 0
VelRight = 0
adjust = 1.9 #4.15
b = 0.095 # distance between Thymio's wheels in m

#Loop tant qu'on est pas au goal final
while((Pablo.x<path[-1][0]-Threshold_d or Pablo.x>path[-1][0] +Threshold_d) and
      (Pablo.y<path[-1][1]-Threshold_d or Pablo.y>path[-1][1] +Threshold_d)):
    th.set_var("motor.right.target", 0)
    th.set_var("motor.left.target", 0)
    a = time.time();
    print(a)
    t = b-a
    SpeedGain = adjust*0.0325 * t

    DTheta = (VelLeft*SpeedGain - VelRight*SpeedGain)/(2*b)
    DS = (VelLeft + VelRight)/2
    
    Pablo.x = Pablo.x+DS*SpeedGain*math.sin(math.radians(Pablo.theta))
    Pablo.y = Pablo.y + DS*SpeedGain*math.cos(math.radians(Pablo.theta))
    Pablo.theta = Pablo.theta+DTheta
    
    #Correction orientation en temps réel si pas arrivé
    if ((Pablo.x < current_goal[0] - Threshold_d or Pablo.x > current_goal[0] + Threshold_d) or
        (Pablo.y < current_goal[1] - Threshold_d or Pablo.y > current_goal[1] + Threshold_d)):
        theta = theta_needed(p_x, p_y, Pablo.x, Pablo.y)
        p_x = Pablo.x
        p_y = Pablo.y
        if(Pablo.theta < theta - Threshold_theta):
            print('turn R')
            th.set_var("motor.right.target", 100)
            th.set_var("motor.left.target", 2**16 - 100)
            b = time.time()
            VelRight = 100
            VelLeft = -100
        elif(Pablo.theta > theta + Threshold_theta):
            print(Pablo.theta)
            print(theta + Threshold_theta)
            th.set_var("motor.right.target", 2**16 - 100)
            th.set_var("motor.left.target", 100)
            b = time.time()
            print(b-a)
            VelRight = -100
            VelLeft = 100
        else:
            print('Forward')
            th.set_var("motor.right.target", 100)
            th.set_var("motor.left.target", 100)
            b = time.time()
            VelRight = 100
            VelLeft = 100


    elif ((Pablo.x >= current_goal[0] - Threshold_d and Pablo.x <= current_goal[0] + Threshold_d) and
          (Pablo.y >= current_goal[1] - Threshold_d and Pablo.y <= current_goal[1] + Threshold_d)):
            #On vide la liste au fur et à mesure qu'on fait les déplacements + conversion en cm
            current_goal = path[0]
            del path[0]
            current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
            b = time.time()
            VelRight = 0
            VelLeft = 0

            #Si on chope la même pos, on prend la suivante
            while (Pablo.x >= current_goal[0] - Threshold_d and Pablo.x <= current_goal[0] + Threshold_d and
                   Pablo.y >= current_goal[1] - Threshold_d and Pablo.y <= current_goal[1] + Threshold_d):
                current_goal = path[0]
                del path[0]
                current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
                b = time.time()
                VelRight = 0
                VelLeft = 0
    time.sleep(1 * 10**(-3)) # wait 1ms before relooping


th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

Enter the order you want to to see the paintings in :1 0
1575728907.78652481575728907.7865248

3.1415926535897933.141592653589793

11

0.0079655647277832030.007965564727783203

1575728907.79947661575728907.7994766

3.14159265357025273.1415926535702527

11

0.0149729251861572270.014972925186157227

1575728907.82141851575728907.8214185

3.14159265354294263.1415926535429426

11

0.0129666328430175780.012966632843017578

1575728907.83937031575728907.8393703

3.1415926535234073.141592653523407

11

0.0099732875823974610.009973287582397461

1575728907.854331575728907.85433

3.1415926535038663.141592653503866

11

0.0247278213500976560.024727821350097656

1575728907.90000181575728907.9000018

3.14159265342179063.1415926534217906

11

0.0139622688293457030.013962268829345703

1575728907.9199481575728907.919948

3.141592653398343.14159265339834

11

0.0099751949310302730.009975194931030273

1575728907.9339111575728907.933911

3.14159265338271253.1415926533827125

11

0.0111742019653320310.01117

1575728908.6940261575728908.694026

3.14159265235065853.1415926523506585

11

0.0057592391967773440.005759239196777344

1575728908.70377371575728908.7037737

3.14159265233502843.1415926523350284

11

0.0068902969360351560.006890296936035156

1575728908.71365551575728908.7136555

3.14159265232330533.1415926523233053

11

0.0136964321136474610.013696432113647461

1575728908.7313251575728908.731325

3.1415926523077363.141592652307736

11

0.0112922191619873050.011292219161987305

1575728908.74630671575728908.7463067

3.14159265229327733.1415926522932773

11

0.0069818496704101560.006981849670410156

1575728908.757611575728908.75761

3.1415926522763423.141592652276342

11

0.0069851875305175780.006985187530517578

1575728908.76852561575728908.7685256

3.14159265226093963.1415926522609396

11

0.0068118572235107420.006811857223510742

1575728908.77996251575728908.7799625

3.14159265224281463.1415926522428146

11

0.0056631565093994140.005663156509399414

1575728908.78840421575728908.7884042

0.00757956504821777340.0075795650482177734

1575728909.7547181575728909.754718

3.1415926508866443.141592650886644

11

0.0125868320465087890.012586832046508789

1575728909.771311575728909.77131

3.14159265087094843.1415926508709484

11

0.00798034667968750.0079803466796875

1575728909.7903551575728909.790355

3.14159265082758843.1415926508275884

11

0.0067689418792724610.006768941879272461

1575728909.80650761575728909.8065076

3.14159265079081563.1415926507908156

11

0.0066349506378173830.006634950637817383

1575728909.81713081575728909.8171308

3.14159265077518633.1415926507751863

11

0.0152125358581542970.015212535858154297

1575728909.83732561575728909.8373256

3.1415926507556623.141592650755662

11

0.0115063190460205080.011506319046020508

1575728909.8525691575728909.852569

3.14159265074101633.1415926507410163

11

0.0059862136840820310.005986213684082031

1575728909.86159871575728909.8615987

3.141592650729093.14159265072909

11

0.0118627548217773440.011862754821777344

15

0.008807420730590820.00880742073059082

1575728910.67466331575728910.6746633

3.14159264945121873.1415926494512187

11

0.0135931968688964840.013593196868896484

1575728910.69229221575728910.6922922

3.14159264943540343.1415926494354034

11

0.011920213699340820.01192021369934082

1575728910.7082021575728910.708202

3.14159264941976923.1415926494197692

11

0.006001234054565430.00600123405456543

1575728910.72696881575728910.7269688

3.1415926493697433.141592649369743

11

0.0059831142425537110.005983114242553711

1575728910.7422861575728910.742286

3.14159264933316433.1415926493331643

11

0.0086185932159423830.008618593215942383

1575728910.75639961575728910.7563996

3.141592649311633.14159264931163

11

0.0114600658416748050.011460065841674805

1575728910.77288411575728910.7728841

3.141592649291943.14159264929194

11

0.014921665191650390.01492166519165039

1575728910.79384021575728910.7938402

3.1415926492682923.141592649268292

11

0.0224728584289550780.022472858428955078

157572


11

0.00647711753845214840.0064771175384521484

1575728911.64695361575728911.6469536

3.1415926480354873.141592648035487

11

0.0059835910797119140.005983591079711914

1575728911.65592981575728911.6559298

3.14159264802375973.1415926480237597

11

0.00623345375061035160.0062334537506103516

1575728911.66615151575728911.6661515

3.14159264800813043.1415926480081304

11

0.0069823265075683590.006982326507568359

1575728911.67712261575728911.6771226

3.14159264799249943.1415926479924994

11

0.0059838294982910160.005983829498291016

1575728911.68709731575728911.6870973

3.141592647976863.14159264797686

11

0.0060527324676513670.006052732467651367

1575728911.6971231575728911.697123

3.14159264796129063.1415926479612906

11

0.0049335956573486330.004933595657348633

1575728911.7070431575728911.707043

3.141592647941753.14159264794175

11

0.0059857368469238280.005985736846923828

1575728911.71701651575728911.7170165

3.14159264792612273.1415926479261227

11

0.00609779357910156250.006097

...

3.14159262872042173.1415926260068407

11

0.0069291591644287110.005983591079711914

1575728924.63856221575728926.4071543

3.14159262870478973.1415926259912013

11

0.0050053596496582030.00897669792175293

1575728924.64856581575728926.4201674

3.14159262868520273.1415926259753832

11

0.0079479217529296880.01048135757446289

1575728924.65950541575728926.4376304

3.14159262867347883.1415926259480234

11

0.0049872398376464840.006980180740356445

1575728924.6675361575728926.448601

3.14159262866155233.1415926259323856

11

0.0049340724945068360.005983829498291016

1575728924.67646151575728926.4595714

3.1415926286459113.1415926259128444

11

0.0059835910797119140.004987239837646484

1575728924.68543741575728926.4695451

3.14159262863418443.141592625893303

11

0.0059843063354492190.004992961883544922

1575728924.69640731575728926.4807467

3.14159262861464673.1415926258689724

11

0.00698065757751464840.004755973815917969

1575728924.70638081575728926.4914863

3.14159262860291833.141592625

1575728925.3222791575728927.0607448

3.1415926276886173.1415926249880273

11

0.0049862861633300780.004986286163330078

1575728925.33165741575728927.073718

3.1415926276714053.1415926249567274

11

0.00558066368103027340.00571441650390625

1575728925.34122921575728927.0826857

3.14159262765576443.1415926249439785

11

0.0050189495086669920.0039882659912109375

1575728925.34920671575728927.0906641

3.14159262764417063.1415926249283417

11

0.0049870014190673830.004987239837646484

1575728925.3581821575728927.0986774

3.14159262762854133.141592624916483

11

0.0049872398376464840.005709171295166016

1575728925.36651181575728927.1076195

3.1415926276154423.1415926249038137

11

0.0076284408569335940.004987239837646484

1575728925.37816931575728927.1166444

3.1415926275996533.141592624887991

11

0.0099737644195556640.004936695098876953

1575728925.39209271575728927.1255715

3.1415926275841753.141592624872353

11

0.00498604774475097660.005525112152099609

1575728925.40130191575728927.1344

0.0099735260009765620.004324674606323242

1575728926.01191741575728927.6661112

3.14159262663284763.1415926240220755

11

0.0060675144195556640.004686594009399414

1575728926.0213331575728927.6737413

3.1415926266197273.1415926240105403

11575728929.7351415

0.0061030387878417973.1415926208514104

1575728926.0322971

3.1415926266006780.005625486373901367

11575728929.7450209

0.0049858093261718753.1415926208347402

1575728926.04227021

3.1415926265811330.005648374557495117

11575728929.7556658

0.006980895996093753.14159262081516

1575728926.0532411

3.1415926265654970.007967710494995117

11575728929.76862

0.0070254802703857423.1415926207956186

1575728926.06421141

3.1415926265500380.004986286163330078

11575728929.778593

0.0052545070648193363.1415926207760765

1575728926.0741851

3.14159262653154460.008976459503173828

11575728929.7915897

0.0059843063354492193.141592620760322

1575728926.08515521

3.1415926265120050.0069501399993896484

11575728929.8015318

0.0059838294982910163.1

1575728930.50396181575728929.8983274

3.141592619646433.1415926206318305

11

0.004626512527465820.007004499435424805

1575728930.51257781575728929.9084506

3.14159261963079573.1415926206196088

11

0.005984544754028320.004865407943725586

1575728930.52281241575728929.9172773

3.14159261961414063.141592620604085

11

0.0067198276519775390.00897526741027832

1575728930.53491471575728929.9312391

3.14159261959304733.141592620584544

11

0.0045902729034423830.00598454475402832

1575728930.5435221575728929.9402542

3.14159261957730563.1415926205726676

11

0.0099744796752929690.011928558349609375

1575728930.55814581575728929.95717

3.14159261955908553.1415926205531237

11

0.0063199996948242190.004987955093383789

1575728930.56861831575728929.967162

3.14159261954281233.141592620533514

11

0.0118050575256347660.005060911178588867

1575728930.58441231575728929.9771166

3.14159261952718043.141592620514336

11

0.0069818496704101560.007087230682373047

1575728930.5953851575728929.9874473

3

0.00698113441467285160.004985809326171875

1575728931.23386861575728930.6881285

3.14159261857659143.1415926193533195

11

0.0049870014190673830.007246494293212891

1575728931.24190281575728930.699099

3.141592618564653.1415926193387254

11

0.0083212852478027340.007979393005371094

1575728931.2542131575728930.7163112

3.1415926185490183.1415926193025436

11

0.0102748870849609380.01033163070678711

1575728931.26817541575728930.731325

3.14159261853456773.1415926192841956

11

0.0049867630004882810.005674600601196289

1575728931.2772071575728930.739989

3.1415926185187173.14159261927248

11

0.00572228431701660160.006035804748535156

1575728931.2863131575728930.7499893

3.1415926185054563.141592619256944

11

0.0067951679229736330.005957365036010742

1575728931.29809671575728930.7589376

3.1415926184859073.1415926192452233

11

0.0079767704010009770.005985260009765625

1575728931.31032731575728930.7689123

3.1415926184692373.141592619229589

11

0.0097093582153320310.007271051406860351

TODO
check les trucs publics et privés de la classe Thymoio

attention angles de move en radians !
soit tout degrés soit rad